In [1]:
import pandas as pd 
import spacy 
import json 
from pandas.io.json import json_normalize
from tqdm import tqdm 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import string


In [2]:
# Generate question data
extracted_rows = []
neg_examples = []


# question text file into csv
with open("train_5500.label.txt", "r", encoding="ISO-8859-1") as f:
    i = 0
    for l in f.readlines():
        split_line = l.split()
        extracted_rows.append({"text": " ".join(split_line[1:]), "label":1})
pos_example = pd.DataFrame(extracted_rows)

# statement & question text file into csv
nlp = spacy.load("en_core_web_sm")
data = json.load(open("train-v2.0.json", "r"))
data.keys()

def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch.lower() for ch in text if ch not in exclude)

import swifter
import spacy 

nlp = spacy.load('en_core_web_sm')

def clean_str(text: str) -> str:
    return " ".join([tok.lemma_ for tok in nlp(text)])

for article in tqdm(data['data']):
    for paragraph in article['paragraphs']:
        for sent in nlp(paragraph['context']).sents:
            extracted_rows.append({"text": sent.text, "label": 0})
        for i, question in enumerate(paragraph['qas']):
            if i % 3 == 0:
                extracted_rows.append({"text": remove_punc(question['question']), "label": 1})
            else:
                extracted_rows.append({"text": question['question'], "label": 1})

for i in tqdm(range(0, len(extracted_rows))):
    extracted_rows[i]['text'] = clean_str(extracted_rows[i]['text'])
    
# statement text file into csv            
data2 = json.load(open("dev-v2.0.json", "r"))

for article in tqdm(data2['data']):
    for paragraph in article['paragraphs']:
        for sent in nlp(paragraph['context']).sents:
            extracted_rows.append({"text": sent.text, "label": 0})


table = pd.DataFrame(extracted_rows)
table.to_csv("qa.csv")

 22%|██▏       | 98/442 [01:40<05:51,  1.02s/it]


KeyboardInterrupt: 

In [4]:
table.groupby('label').count()

NameError: name 'table' is not defined

In [2]:
# make a model 
train, test = train_test_split(table, stratify = table['label'], random_state = 75)

tfidf = TfidfVectorizer()
tfidf.fit(table['text'])

X_train = tfidf.transform(train['text'])
X_test = tfidf.transform(test['text'])

print(train.groupby(['label']).count())
test.groupby(['label']).count()

NameError: name 'train_test_split' is not defined

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# multinomial fitting 
clf = MultinomialNB(alpha = 0.6)
clf.fit(X_train, train['label'])
clf.score(X_train, train['label'])

mpreds = clf.predict(X_test)
print(classification_report(test['label'], mpreds))

              precision    recall  f1-score   support

           0       0.92      0.76      0.83     26036
           1       0.83      0.95      0.89     32580

    accuracy                           0.86     58616
   macro avg       0.88      0.85      0.86     58616
weighted avg       0.87      0.86      0.86     58616



In [8]:
from sklearn.linear_model import LogisticRegression

# logistic regression fitting
lclf = LogisticRegression()
lclf.fit(X_train, train['label'])
lclf.score(X_train, train['label'])

lpreds = lclf.predict(X_test)
print(classification_report(test['label'], lpreds))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98     26036
           1       0.98      0.98      0.98     32580

    accuracy                           0.98     58616
   macro avg       0.98      0.98      0.98     58616
weighted avg       0.98      0.98      0.98     58616



In [1]:
# Linear Regression 
from sklearn.linear_model import LinearRegression

lrclf = LinearRegression()
lrclf.fit(X_train, train['label'])
lrclf.score(X_train, train['label'])

lrpreds = lrclf.predict(X_test)
print(classification_report(test['label'], lrpreds))

NameError: name 'X_train' is not defined

In [9]:
# Random Forest Classifier 
from sklearn.ensemble import RandomForestClassifier

rclf = RandomForestClassifier()
rclf.fit(X_train, train['label'])
rclf.score(X_train, train['label'])

rpreds = rclf.predict(X_test)
print(classification_report(test['label'], rpreds))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95     26036
           1       0.96      0.96      0.96     32580

    accuracy                           0.96     58616
   macro avg       0.96      0.96      0.96     58616
weighted avg       0.96      0.96      0.96     58616



In [10]:
import csv
import numpy as np
from sklearn.metrics import classification_report

# apply model to posh data
message = pd.read_csv("all_user_questions.csv")
message['message'] = message['message'].apply(lambda x: str(x)) 

m_encoded = tfidf.transform(message['message'].tolist())

# get predictions and scores
preds = lclf.predict(m_encoded)
scores = lclf.predict_proba(m_encoded)
 
message["is_question"] = preds
message['is_question_score'] = np.max(scores, axis = 1)

# get random sample and export to csv
m2 = message.sample(n=300, random_state = 2000)
m2.to_csv("sample.csv")



In [11]:
# get classification report
ms = pd.read_csv("sample2.csv")
print(classification_report(ms['is_question'], ms['gold_score']))

              precision    recall  f1-score   support

           0       0.96      0.84      0.90       181
           1       0.80      0.95      0.87       119

    accuracy                           0.88       300
   macro avg       0.88      0.89      0.88       300
weighted avg       0.90      0.88      0.88       300



In [12]:
import csv
import numpy as np
from sklearn.metrics import classification_report

# apply model to posh data
message = pd.read_csv("all_user_questions.csv")
message['message'] = message['message'].apply(lambda x: str(x)) 

m_encoded = tfidf.transform(message['message'].tolist())

# get predictions and scores
preds = rclf.predict(m_encoded)
scores = rclf.predict_proba(m_encoded)
 
message["is_question"] = preds
message['is_question_score'] = np.max(scores, axis = 1)

# get random sample and export to csv
m2 = message.sample(n=300, random_state = 2000)
m2.to_csv("rsample.csv")



In [13]:
ms = pd.read_csv("rsample2.csv")
print(classification_report(ms['is_question'], ms['gold_score']))

              precision    recall  f1-score   support

           0       0.85      0.83      0.84       162
           1       0.81      0.83      0.82       138

    accuracy                           0.83       300
   macro avg       0.83      0.83      0.83       300
weighted avg       0.83      0.83      0.83       300



In [92]:
for i in tqdm(range(0, len(extracted_rows))):
    extracted_rows[i]['text'] = clean_str(extracted_rows[i]['text'])

table = pd.DataFrame(extracted_rows)
table.to_csv("qa.csv")

100%|██████████| 234463/234463 [28:32<00:00, 136.88it/s] 


In [22]:
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression

model = SentenceTransformer('bert-base-nli-mean-tokens')

X_train_bert = model.encode(train['text'].tolist(), show_progress_bar = True)
Y_train_bert = train['label']


Batches: 100%|██████████| 21981/21981 [2:10:33<00:00,  2.81it/s]  


In [25]:
# logistic regression fitting
rclf = RandomForestClassifier(verbose = True, n_jobs = -1)
rclf.fit(X_train_bert, Y_train_bert)

X_test_bert = model.encode(test['text'].tolist(), show_progress_bar = True)
Y_test_bert = test['label']

print(classification_report(Y_test_bert, rclf.predict(X_test_bert)))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   48.8s finished
Batches: 100%|██████████| 7327/7327 [38:40<00:00,  3.16it/s]


NameError: name 'y_test_bert' is not defined

In [26]:
print(classification_report(Y_test_bert, rclf.predict(X_test_bert)))

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.1s finished


              precision    recall  f1-score   support

           0       0.88      0.92      0.90     26036
           1       0.94      0.90      0.92     32580

    accuracy                           0.91     58616
   macro avg       0.91      0.91      0.91     58616
weighted avg       0.91      0.91      0.91     58616

